In [319]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gp
from pandas import DataFrame
import math
import seaborn as sns
%pylab inline

Populating the interactive namespace from numpy and matplotlib


#PART 1
## Get a picture of today values in our study area

## Ridership and foot traffic analysis:

In [320]:
# pull in files from turnstile notebook
metropolitan = pd.read_csv('station_metropolitan.csv')
lorimer = pd.read_csv('station_lorimer.csv')
nassau = pd.read_csv('station_nassau.csv')
bedford = pd.read_csv('station_bedford.csv')
graham = pd.read_csv('station_graham.csv')

In [321]:
Dictionary_stations={'metropolitan':metropolitan, 'lorimer':lorimer,'nassau':nassau,'bedford':bedford,'graham':graham}

In [322]:
av_daily_exits=pd.DataFrame(index=Dictionary_stations.keys(),columns=['count_daily_exits'])
for st in Dictionary_stations:
    station=Dictionary_stations[st]
    station.rename(columns = {'Unnamed: 0':'turnstile'}, inplace = True) 
    station.set_index('turnstile',inplace = True)
    for date in station.columns:
        values=station[date]
        z_test=abs((values-mean(values)))<2.5*std(values)
        outliers_indexes=z_test.index[z_test==False]
        if len(outliers_indexes!=0):
            print('OUTLIERS:',outliers_indexes,date,st)
            # replace the anomaly value with the median of that day
            station.replace(station.loc[outliers_indexes,date][0], np.median(station[date]), inplace = True)
    totals_list = [station[i].sum() for i in station.columns]
    av_daily_exits.count_daily_exits[st]=int(np.mean(totals_list))
    
    

('OUTLIERS:', Index([u'R35300-06-00'], dtype='object', name=u'turnstile'), '02/12/2016', 'lorimer')
('OUTLIERS:', Index([u'R35300-00-01'], dtype='object', name=u'turnstile'), '02/25/2016', 'lorimer')
('OUTLIERS:', Index([u'R23500-06-01'], dtype='object', name=u'turnstile'), '02/16/2016', 'bedford')


In [323]:
av_daily_exits['percent_daily_exits']=(av_daily_exits.count_daily_exits/np.sum(av_daily_exits.count_daily_exits))*100
av_daily_exits

,count_daily_exits,percent_daily_exits
nassau,3807,9.085268
lorimer,2553,6.092643
graham,5660,13.50739
metropolitan,3939,9.400282
bedford,25944,61.91442


Now, we are going to estimate the foot traffic in the area around Bedford Avenue. For that estimate, we consider a portion of the exits (expansion weight) on the other stations and the total of people comming out from Bedford.

We chose the negative exponencial function to model this expansion: this function has the value of 1 when the distance is 0 (that means, the weight is 1 in Bedford av) and the value vanishes quickly as the distance increases. 

In [324]:
###GRAFIC OF THIS FUNCTION!!

In [325]:
distances_dic={'lorimer': 0.42*5280,
'graham':0.7*5280,
'metropolitan': 0.41*5280 ,
'nassau': 0.58*5280,
'bedford':0              }
distances=pd.DataFrame.from_dict(distances_dic,orient='index')
distances.rename(columns={0:'distance_to_bedford'},inplace= True)
expansion_coef=1500
distances['expansion_proportion']=np.exp(-distances.distance_to_bedford/expansion_coef)
foot_traffic=av_daily_exits.join(distances)
foot_traffic

,count_daily_exits,percent_daily_exits,distance_to_bedford,expansion_proportion
nassau,3807,9.085268,3062.4,0.129821
lorimer,2553,6.092643,2217.6,0.228002
graham,5660,13.50739,3696.0,0.085094
metropolitan,3939,9.400282,2164.8,0.236171
bedford,25944,61.91442,0.0,1.000000


In [326]:
##GRAPHIC WITH DIFFERENT EXPANSION COEFFICIENTS

Then, the foot traffic is estimated as follows:

In [327]:
R_1=sum(foot_traffic.expansion_proportion*foot_traffic.count_daily_exits)

In [328]:
print('CONCLUSION: The total daily ridership today in the area around Bedford Av. station is equal to R=%d')%R_1

CONCLUSION: The total daily ridership today in the area around Bedford Av. station is equal to R=28432


##Expenditure in Williamsburgh

- Total households in three Williamsburg zip codes (11206, 11211, and 11237)
- Average household expenture in Williamsburg
- Spend by category (listed below) that could contribute to GDP (categorical breakouts only available for all of NYC)

In [329]:
# households and average household anual expenditures for williamsburg zipcodes
# taken from http://www.point2homes.com/US/Neighborhood/NY/Brooklyn/Williamsburg-Demographics.html
households = 68787
total_expend = 40514

In [330]:
# estimate regional expenditure ammount 
Exp_wil = households * total_expend
print 'Estimated Total Household Expenditures in Williamsburg is: $',Exp_wil

Estimated Total Household Expenditures in Williamsburg is: $ 2786836518


However, we are interested in expenditures that might be associated with commercial activity in the area (for example: education is not associated with commercial activity). 
We are concidering expenditures in the areas of: 
 health = .056
 
 entertain = .042 
 
 apparel = .039 
 
 dining = .132 
 
 other = .058

In [331]:
# health = .056
# entertain = .042 
# apparel = .039 
# dining = .132 
# other = .058

category_names = ['healthcare', 'entertainment', 'apparel', 'dining','other']
categories = [.056, .042, .039, .132, .058]
my_list = [i*Exp_wil for i in categories]
#Commercial expend:
spend = pd.DataFrame(my_list, index= category_names)
spend.rename(columns = {0:'Spend $'}, inplace = True)
spend['Spend %'] = (spend['Spend $']/np.sum(spend['Spend $']))*100
spend

,Spend $,Spend %
healthcare,1.560628e+08,17.125382
entertainment,1.170471e+08,12.844037
apparel,1.086866e+08,11.926606
dining,3.678624e+08,40.366972
other,1.616365e+08,17.737003


In [332]:
# total relevant expenses ammount is just the sum of all relevant categories
com_spend_W = sum(spend['Spend $'])
print 'Estimated Commercial spend in Williamsburg is: $',int(com_spend_W)
print 'Estimated Commercial spend is', (com_spend_W/Exp_wil)*100,'% of total estimated expenditures.'

Estimated Commercial spend in Williamsburg is: $ 911295541
Estimated Commercial spend is 32.7 % of total estimated expenditures.


## Calculate the amount of commercial space around the Bedford Ave. station
- use PLUTO data
- Total commercial space for same three Williamsburg zip codes (11206, 11211, 11237)
- Commercial space within radius around the Bedford Ave. station, calculated using ArcGIS

In [333]:
#This file contains PLUTO data aggregated by census blocks around Bedford Av.
pluto_df=pd.read_csv('pluto_lehd_blocks.csv')
pluto_df.head()

,FID,bldgar,comare,resare,retail,office,zipco,geocode,C1,C2,C3,C4
0,0,456906,54500,402406,8000,28325,11211,360470517001001,40,6,0,0
1,1,180591,65390,115201,24730,20000,11211,360470517001002,82,133,1,0
2,2,227160,92068,135092,46914,32111,11211,360470517001003,227,77,0,5
3,3,100641,15010,85631,7440,6270,11211,360470517001004,20,159,0,0
4,4,156414,54939,101475,29310,0,11249,360470517002001,55,114,0,0


In [334]:
#Commercial Density
dC=(pluto_df['comare']+pluto_df['retail'])/pluto_df['bldgar']
pluto_df['commercial_density']=dC

In [335]:
ct_area=np.sum(pluto_df.comare)
# PLUTO: total Commerce area in Williamsburgh:
total_comm = 39715979.0
target_area = ct_area/total_comm

print 'Commercial square footage within radius is:', ct_area
print 'Total commercial square footage in Williamsburg is:', int(total_comm)
print 'The percentage of square footage within radius is:', target_area*100,'%'

Commercial square footage within radius is: 2595120
Total commercial square footage in Williamsburg is: 39715979
The percentage of square footage within radius is: 6.53419622364 %


In [336]:
# calculate commercial spend in our area, 
com_exp_bed = target_area * com_spend_W
com_exp_bed
print 'The estimated size of our economy around the Bedford Station is: $', int(com_exp_bed)

The estimated size of our economy around the Bedford Station is: $ 59545838


##Relationship Between Foot Traffic and Density of Comercial Areas

In [337]:
pluto_df.head()

,FID,bldgar,comare,resare,retail,office,zipco,geocode,C1,C2,C3,C4,commercial_density
0,0,456906,54500,402406,8000,28325,11211,360470517001001,40,6,0,0,0.136790
1,1,180591,65390,115201,24730,20000,11211,360470517001002,82,133,1,0,0.499028
2,2,227160,92068,135092,46914,32111,11211,360470517001003,227,77,0,5,0.611824
3,3,100641,15010,85631,7440,6270,11211,360470517001004,20,159,0,0,0.223070
4,4,156414,54939,101475,29310,0,11249,360470517002001,55,114,0,0,0.538628


In [338]:
pluto_df['alpha_coef']=pluto_df.commercial_density/(R_1/1000)
print "relationship between comemrcial_density and foot traffic", mean(pluto_df.alpha_coef)

relationship between comemrcial_density and foot traffic 0.0157368720692


#PART 1
## Estimating impact on foot traffic

LEHD Data gave us that the proportion of workers that might be affected for the Canarsie tunel shotdown (people traveling from Brooklyn to Manhattan everyday) is 58%. 

So now, we are going to assume that our foot traffic is going to decrease for Bedford avenue to a 42% of the original value.

In [339]:
#impact_on_traf=0.58
impact_on_traf=(3.0/2.0)*0.58

foot_traffic2=foot_traffic
foot_traffic2.head()
foot_traffic2.loc['bedford','count_daily_exits']=int(foot_traffic.loc['bedford','count_daily_exits']*impact_on_traf)
foot_traffic2

,count_daily_exits,percent_daily_exits,distance_to_bedford,expansion_proportion
nassau,3807,9.085268,3062.4,0.129821
lorimer,2553,6.092643,2217.6,0.228002
graham,5660,13.50739,3696.0,0.085094
metropolitan,3939,9.400282,2164.8,0.236171
bedford,22571,61.91442,0.0,1.000000


In [340]:
R_2=sum(foot_traffic.expansion_proportion*foot_traffic.count_daily_exits)
print('CONCLUSION: The torecasted daily foot traffic in the area around Bedford Av. station is equal to R=%d')%R_2

CONCLUSION: The torecasted daily foot traffic in the area around Bedford Av. station is equal to R=25059


## Estimated Impact on Commercial Areas

Previously, the alpha coefficient (relationship between commercial density and foot traffic) was calculated for each census block.
Now, we are going to use the same alpha coefficient to calculate the possible reduction on commercial density

In [341]:
pluto_df2=pluto_df

In [342]:
pluto_df2['projected_commercial_density']=pluto_df['commercial_density']*(R_2/1000)*pluto_df['alpha_coef']
pluto_df2['projected_commercial_area']=pluto_df2['projected_commercial_density']*pluto_df2['bldgar']
pluto_df2.head()

,FID,bldgar,comare,resare,retail,office,zipco,geocode,C1,C2,C3,C4,commercial_density,alpha_coef,projected_commercial_density,projected_commercial_area
0,0,456906,54500,402406,8000,28325,11211,360470517001001,40,6,0,0,0.136790,0.004811,0.016492,7535.116362
1,1,180591,65390,115201,24730,20000,11211,360470517001002,82,133,1,0,0.499028,0.017551,0.219486,39637.207760
2,2,227160,92068,135092,46914,32111,11211,360470517001003,227,77,0,5,0.611824,0.021519,0.329921,74944.892997
3,3,100641,15010,85631,7440,6270,11211,360470517001004,20,159,0,0,0.223070,0.007846,0.043857,4413.819175
4,4,156414,54939,101475,29310,0,11249,360470517002001,55,114,0,0,0.538628,0.018944,0.255703,39995.458215


In [347]:
ct_area=np.sum(pluto_df.comare)
ct_area2=np.sum(pluto_df2.projected_commercial_area)
# PLUTO: total Commerce area in Williamsburgh:


print 'Commercial square footage within radius today is:', ct_area
print'Commercial square footage within radius projected is:', ct_area2
print 'decrease of',(1-ct_area2/ct_area)

Commercial square footage within radius today is: 2595120
Commercial square footage within radius projected is: 2353232.1982
decrease of 0.093208715511


In [348]:
# calculate commercial spend in our area, 
total_comm = 39715979.0
target_area2 = ct_area2/total_comm
com_exp_bed_proj = target_area2 * com_spend_W
com_exp_bed=target_area * com_spend_W
print 'The estimated size of our economy around the Bedford Station is: $', int(com_exp_bed)
print 'The estimated size of our economy around the Bedford Station after closure of tunel: $', int(com_exp_bed_proj)
print 'decrease of',(1-com_exp_bed_proj/com_exp_bed)

The estimated size of our economy around the Bedford Station is: $ 59545838
The estimated size of our economy around the Bedford Station after closure of tunel: $ 53995647
decrease of 0.093208715511
